In [45]:
import pandas as pd
import cfbd
from cfbd.rest import ApiException

# Configure API key authorization
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'aiS0bBk23RvX4eOWirh4JQgCK+UIO0k1AF/9A2RDyu2D6D/u1LLVKvdDKdHQokqF'
configuration.api_key_prefix['Authorization'] = 'Bearer'

# Create an instance of the API class
api_instance = cfbd.PlayersApi(cfbd.ApiClient(configuration))

try:
    # Fetch transfer portal data for the current year
    api_response = api_instance.get_transfer_portal(year=2023)  # Replace 2023 with the desired year if needed
    
    # Convert the response to a DataFrame
    df = pd.DataFrame([player.to_dict() for player in api_response])
    print(df.head())  # You can use df to further analyze or process the transfer data
except ApiException as e:
    print("Exception when calling PlayersApi->get_transfer_portal: %s\n" % e)

# Rename columns to proper caps
df = df.rename(columns={'season': 'Season',
                        'first_name': 'First Name',
                        'last_name': 'Last Name',
                        'position': 'Position',
                        'origin': 'Origin School',
                        'destination': 'Destination School',
                        'transfer_date': 'Transfer Date',
                        'rating': 'Rating',
                        'stars': 'Stars',
                        'eligibility': 'Eligibility'})

df.to_csv('data/portal.csv', index=False)
df.head()

   season first_name  last_name position                 origin  \
0    2023     Joshua     Farmer       DL          Florida State   
1    2023      A'Jon     Vivens       RB         Colorado State   
2    2023      Jamar      Rucks       DL      Appalachian State   
3    2023       Nate  Jefferson       WR  Florida International   
4    2023      Aaron    Bedgood       WR       Coastal Carolina   

      destination             transfer_date  rating  stars eligibility  
0            None  2023-04-27T17:23:00.000Z     NaN    3.0   Immediate  
1            None  2022-10-31T21:23:00.000Z     NaN    3.0   Immediate  
2  Kennesaw State  2022-12-05T11:46:00.000Z     NaN    3.0   Immediate  
3            None  2023-07-06T22:31:00.000Z     NaN    3.0   Immediate  
4            None  2023-07-06T18:38:00.000Z     NaN    2.0   Immediate  


,Season,First Name,Last Name,Position,Origin School,Destination School,Transfer Date,Rating,Stars,Eligibility
0,2023,Joshua,Farmer,DL,Florida State,None,2023-04-27T17:23:00.000Z,NaN,3.0,Immediate
1,2023,A'Jon,Vivens,RB,Colorado State,None,2022-10-31T21:23:00.000Z,NaN,3.0,Immediate
2,2023,Jamar,Rucks,DL,Appalachian State,Kennesaw State,2022-12-05T11:46:00.000Z,NaN,3.0,Immediate
3,2023,Nate,Jefferson,WR,Florida International,None,2023-07-06T22:31:00.000Z,NaN,3.0,Immediate
4,2023,Aaron,Bedgood,WR,Coastal Carolina,None,2023-07-06T18:38:00.000Z,NaN,2.0,Immediate


In [14]:
import dash
from dash import dcc, html, Input, Output, dash_table
import pandas as pd

# Initialize the Dash app
app = dash.Dash(__name__)

df = pd.read_csv('data/portal.csv')

# Unique list of schools
schools = df['Origin School'].dropna().unique().tolist() + df['Destination School'].dropna().unique().tolist()
schools = list(set([school for school in schools if school]))  # Removing duplicates and None

app.layout = html.Div([

     html.H3("2023 Transfer Portal - Whose Coming and Going?"),

    # Image container
    html.Div(
        html.Img(src='/assets/Portal.png', width='300px', height='auto')),

    # Dropdown container
    html.Div(
        dcc.Dropdown(
            id='school-dropdown',
            options=[{'label': school, 'value': school} for school in schools],
            value=schools[0],  # Default value
        ),
        style={'width': '25%', 'margin': '10px'}
    ),

    # Table for 'Who is Coming'
    html.Div(
        [html.H3("Who is Coming"),
        dash_table.DataTable(id='joining-players-table')],
        style={'width': '60%', 'margin': '10px'}
    ),

    # Table for 'Who is Leaving'
    html.Div(
        [html.H3("Who is Leaving"),
        dash_table.DataTable(id='leaving-players-table')],
        style={'width': '60%', 'margin': '10px'}
    ),

    # Table for 'Average Ratings and Stars'
    html.Div(
        [html.H3("Average Ratings and Stars"),
        dash_table.DataTable(id='avg-ratings-table')],
        style={'width': '60%', 'margin': '10px'}
    ),
], style={'display': 'flex', 'flex-direction': 'column', 'align-items': 'center'})


# Callback for updating the tables and average ratings
@app.callback(
    [Output('leaving-players-table', 'data'),
     Output('leaving-players-table', 'columns'),
     Output('joining-players-table', 'data'),
     Output('joining-players-table', 'columns'),
     Output('avg-ratings-table', 'data'),
     Output('avg-ratings-table', 'columns')],
    [Input('school-dropdown', 'value')]
)
def update_tables(selected_school):
    # Filter dataframes for leaving and joining players
    df_leaving = df[(df['Origin School'] == selected_school) & (df['Season'] == 2023)].sort_values(by='Stars', ascending=False)
    df_joining = df[(df['Destination School'] == selected_school) & (df['Season'] == 2023)].sort_values(by='Stars', ascending=False)

    # Prepare data for the player tables
    table_data_leaving = df_leaving[['First Name', 'Last Name', 'Origin School','Destination School', 'Rating', 'Stars']].to_dict('records')
    columns_leaving = [{"name": i, "id": i} for i in df_leaving[['First Name', 'Last Name', 'Origin School','Destination School', 'Rating', 'Stars']].columns]
    
    table_data_joining = df_joining[['First Name', 'Last Name', 'Origin School', 'Rating', 'Stars']].to_dict('records')
    columns_joining = [{"name": i, "id": i} for i in df_joining[['First Name', 'Last Name', 'Origin School', 'Rating', 'Stars']].columns]

    # Calculate average ratings and stars
    avg_rating_leaving = df_leaving['Rating'].mean()
    avg_stars_leaving = df_leaving['Stars'].mean()
    avg_rating_joining = df_joining['Rating'].mean()
    avg_stars_joining = df_joining['Stars'].mean()

    # Calculate rating and star scores
    rating_score = avg_rating_joining / avg_rating_leaving if avg_rating_leaving else None
    star_score = avg_stars_joining / avg_stars_leaving if avg_stars_leaving else None

    # Prepare data for the average ratings and stars table
    avg_ratings_data = [
        {
            'Method': 'Rating',
            'Avg Leaving': avg_rating_leaving,
            'Avg Joining': avg_rating_joining,
            'Score': rating_score
        },
        {
            'Method': 'Stars',
            'Avg Leaving': avg_stars_leaving,
            'Avg Joining': avg_stars_joining,
            'Score': star_score
        }
    ]
    avg_ratings_columns = [{"name": i, "id": i} for i in ['Method', 'Avg Leaving', 'Avg Joining', 'Score']]

    return table_data_leaving, columns_leaving, table_data_joining, columns_joining, avg_ratings_data, avg_ratings_columns

if __name__ == '__main__':
    app.run_server(debug=True, port=8052)


In [59]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [10]:
import dash
from dash import dcc, html, Input, Output, dash_table
import pandas as pd

# Initialize the Dash app
app = dash.Dash(__name__)

df = pd.read_csv('data/portal.csv')

# Unique list of schools
schools = df['Origin School'].dropna().unique().tolist() + df['Destination School'].dropna().unique().tolist()
schools = list(set([school for school in schools if school]))  # Removing duplicates and None

app.layout = html.Div([

     html.H3("2023 Transfer Portal - Whose Coming and Going?"),

    # Image container
    html.Div(
        html.Img(src='/assets/Portal.png', width='300px', height='auto')),

    # Dropdown container
    html.Div(
        dcc.Dropdown(
            id='school-dropdown',
            options=[{'label': school, 'value': school} for school in schools],
            value=schools[0],  # Default value
        ),
        style={'width': '25%', 'margin': '10px'}
    ),

    # Table for 'Average Ratings and Stars'
    html.Div(
        [html.H3("Average Ratings and Stars"),
        dash_table.DataTable(id='avg-ratings-table')],
        style={'width': '60%', 'margin': '10px'}
    ),
    
    # Table for 'Who is Coming'
    html.Div(
        [html.H3("Who is Coming?"),
        dash_table.DataTable(id='joining-players-table')],
        style={'width': '75%', 'margin': '10px'}
    ),

    # Table for 'Who is Leaving'
    html.Div(
        [html.H3("Who is Leaving?"),
        dash_table.DataTable(id='leaving-players-table')],
        style={'width': '75%', 'margin': '10px'}
    ),

], style={'display': 'flex', 'flex-direction': 'column', 'align-items': 'center'})


# Callback for updating the tables and average ratings
@app.callback(
    [Output('leaving-players-table', 'data'),
     Output('leaving-players-table', 'columns'),
     Output('joining-players-table', 'data'),
     Output('joining-players-table', 'columns'),
     Output('avg-ratings-table', 'data'),
     Output('avg-ratings-table', 'columns')],
    [Input('school-dropdown', 'value')]
)
def update_tables(selected_school):
    # Filter dataframes for leaving and joining players
    df_leaving = df[(df['Origin School'] == selected_school) & (df['Season'] == 2023)].sort_values(by='Stars', ascending=False)
    df_joining = df[(df['Destination School'] == selected_school) & (df['Season'] == 2023)].sort_values(by='Stars', ascending=False)

    # Prepare data for the player tables
    table_data_leaving = df_leaving[['First Name', 'Last Name', 'Origin School','Destination School', 'Rating', 'Stars']].to_dict('records')
    columns_leaving = [{"name": i, "id": i} for i in df_leaving[['First Name', 'Last Name', 'Origin School','Destination School', 'Rating', 'Stars']].columns]

    table_data_joining = df_joining[['First Name', 'Last Name', 'Origin School', 'Destination School','Rating', 'Stars']].to_dict('records')
    columns_joining = [{"name": i, "id": i} for i in df_joining[['First Name', 'Last Name', 'Origin School', 'Destination School','Rating', 'Stars']].columns]

    # Calculate average ratings and stars
    avg_rating_leaving = df_leaving['Rating'].mean()
    avg_stars_leaving = df_leaving['Stars'].mean()
    avg_rating_joining = df_joining['Rating'].mean()
    avg_stars_joining = df_joining['Stars'].mean()

    # Calculate rating and star scores
    rating_score = avg_rating_joining / avg_rating_leaving if avg_rating_leaving else None
    star_score = avg_stars_joining / avg_stars_leaving if avg_stars_leaving else None

    # Prepare data for the average ratings and stars table
    avg_ratings_data = [
        {
            'Metric': 'Average Joining',
            'Stars': round(avg_stars_joining, 2),
            'Rating': round(avg_rating_joining, 2)
        },
        {
            'Metric': 'Average Leaving',
            'Stars': round(avg_stars_leaving, 2),
            'Rating': round(avg_rating_leaving, 2)
        },
        {
            'Metric': 'Score',
            'Stars': round(star_score, 2) if star_score is not None else None,
            'Rating': round(rating_score, 2) if rating_score is not None else None
        }
    ]

    avg_ratings_columns = [{"name": i, "id": i} for i in ['Metric', 'Stars', 'Rating']]



    return table_data_leaving, columns_leaving, table_data_joining, columns_joining, avg_ratings_data, avg_ratings_columns

if __name__ == '__main__':
    app.run_server(debug=True, port=8080)  # Replace 8080 with any other free port
